In [2]:
import tiktoken

### 2.5 Byte pair encoding

In [3]:
tokenizer = tiktoken.get_encoding("gpt2")

In [4]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
"of someunknownPlace. し"

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

tokenizer.decode(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114]


'Hello, do you like tea? <|endoftext|> In the sunlit terraces'

### 2.6 Data sampling with a sliding window

In [5]:
with open("the-verdict.txt","r",encoding="utf-8") as fp:
    raw_text = fp.read()

enc_text = tokenizer.encode(raw_text)

enc_sample = enc_text[50:] #remove first 50 tokens

context_size = 4
x = enc_sample[:context_size] #0-4
y = enc_sample[1:context_size+1] #1-5

print(f"x: {x}")
print(f"y:     {y}")


x: [290, 4920, 2241, 287]
y:     [4920, 2241, 287, 257]


In [6]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desire = enc_sample[i]
    print(context, "--->", desire)

[290] ---> 4920
[290, 4920] ---> 2241
[290, 4920, 2241] ---> 287
[290, 4920, 2241, 287] ---> 257


In [7]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desire = enc_sample[i]
    print(tokenizer.decode(context), "--->", tokenizer.decode([desire]))

 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text)

        for i in range(0, len(token_ids)-max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]




In [9]:
dataset = GPTDatasetV1(raw_text,tokenizer,5,1)
dataset[0]

(tensor([  40,  367, 2885, 1464, 1807]),
 tensor([ 367, 2885, 1464, 1807, 3619]))

In [10]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader


In [11]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
    raw_text = f.read()


dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [14]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4,shuffle=False
)
data_iter = iter(dataloader)
inputs, target = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", target)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


### Creating token embedding

In [15]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [16]:
input_ids = torch.tensor([2,3,5,1])
embedding_layer(input_ids)

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)

### Encoding word positions

In [25]:
output_dim = 256
vocab_size = 50257

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, stride=4,shuffle=False
)

inputs, target = next(iter(dataloader))

print("Input shape:\n", inputs.shape)

Input shape:
 torch.Size([8, 4])


In [34]:
token_embedding = token_embedding_layer(inputs)
print(token_embedding.shape)

torch.Size([8, 4, 256])


In [30]:
torch.arange(4)

tensor([0, 1, 2, 3])

In [35]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embedding = pos_embedding_layer(torch.arange(context_length))

pos_embedding.shape

torch.Size([4, 256])

In [37]:
inputs_embedding = pos_embedding + token_embedding
inputs_embedding.shape

torch.Size([8, 4, 256])

## Coding attention mechanism